In [1]:
!pip install open_clip_torch sentence-transformers openai-clip
!pip install git+https://github.com/huggingface/transformers
!pip install huggingface_hub
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.5 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=e5e13b5785a27911fb8193e86998a486da31e4e9c00358bf3adaea29f9b72e39
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i07tzpzl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-i07tzpzl
  Resolved https://github.com/huggingface/transformers to commit 3d2900e829ab16757632f9dde891f1947cfc4be0
  Installing build dependen

In [2]:
import torch
import open_clip
import glob
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm
import numpy as np
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from sentence_transformers import util
from tqdm import tqdm
from PIL import Image, ImageOps
from collections import Counter
from numpy.linalg import norm
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from transformers import (CLIPProcessor, 
                          CLIPModel,AutoModel, 
                          TrainingArguments)
import clip
import torch.nn as nn

- Setting

In [23]:
src_dir = '/kaggle/input/image-search/test/images'
query_dir = '/kaggle/input/image-search/queries/queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
submission['dot_class'] = 22
submission['cosine_class'] = 22
submission['euclidean_class'] = 22

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

- Define model

In [6]:
model = AutoModel.from_pretrained('laion/CLIP-ViT-bigG-14-laion2B-39B-b160k').cuda().eval()
processor = CLIPProcessor.from_pretrained('laion/CLIP-ViT-bigG-14-laion2B-39B-b160k')

config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/120k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/169M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

- Querying

In [24]:
with torch.no_grad():
    query_images = []
    query_classes = []
    for file in os.listdir(query_dir):
        inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('L')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        query_images.append(outputs)
        query_classes.append(int(file[:-4]))
    query_images = torch.cat(query_images)
    for idx, row in submission.iterrows():
        if not pd.isna(row['class']):
            continue
        inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('L')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        values = outputs @ query_images.T
        if values.softmax(1).max() > .055:
            submission.at[idx, 'dot_class'] = query_classes[values.argmax().numpy().tolist()]
        cosine = torch.cosine_similarity(outputs, query_images)
        if cosine.max() > 0.7:
            submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().numpy().tolist()]
        euclidean_distances = torch.cdist(outputs, query_images, p=2)
        min_distance, min_idx = euclidean_distances.min(dim=1)
        if min_distance.item() < 0.4:
            closest_class = query_classes[min_idx.item()]
            submission.at[idx, 'euclidean_class'] = closest_class

In [25]:
image_files = submission['img_file'].unique()

voting_results = pd.DataFrame({'img_file': image_files, 'voted_class': None})


for idx, image_file in enumerate(image_files):

    dot_vote = submission.loc[submission['img_file'] == image_file, 'dot_class'].mode().values[0]
    cosine_vote = submission.loc[submission['img_file'] == image_file, 'cosine_class'].mode().values[0]
    euclidean_vote = submission.loc[submission['img_file'] == image_file, 'euclidean_class'].mode().values[0]


    votes = [dot_vote, cosine_vote, euclidean_vote]
    voted_class = max(set(votes), key=votes.count)


    voting_results.loc[voting_results['img_file'] == image_file, 'voted_class'] = voted_class


final_results = voting_results[['img_file', 'voted_class']]
print(final_results.head())

                                   img_file voted_class
0  64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg          22
1  c6df1385-382a-4428-b41e-f2d729b90c87.jpg          22
2  af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg          22
3  3fc8998e-0324-426c-8233-6b76abc7e200.jpg          22
4  309d1085-0d11-4411-9555-b24cc8fcee02.jpg          22


In [26]:
final_results.to_csv('submission.csv',index=False)